In [4]:
import cv2
import base64
import requests
import numpy as np

# 1. AYARLAR
API_KEY = "uwbLCWSnHV2geuphu3jv"
MODEL_ID = "safety-vests/14"
VIDEO_PATH = "test_yetkili.mp4"
CONFIDENCE_THRESHOLD = 0.85  # %75 ve üzeri güven skoruna sahip tahminleri kabul et

# 2. ROBOFLOW API FONKSİYONU
def predict_roboflow(frame):
    retval, buffer = cv2.imencode('.jpg', frame)
    img_str = base64.b64encode(buffer).decode('utf-8')
    url = f"https://detect.roboflow.com/{MODEL_ID}?api_key={API_KEY}"
    response = requests.post(url, data=img_str, headers={"Content-Type": "application/x-www-form-urlencoded"})
    return response.json()

# 3. VİDEO İŞLEME
cap = cv2.VideoCapture(VIDEO_PATH)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('yelek_test_yuksek_threshold.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    if frame_idx % 10 == 0:
        try:
            predictions = predict_roboflow(frame)
            if 'predictions' in predictions:
                for pred in predictions['predictions']:
                    conf = pred['confidence']
                    label = pred['class']

                    # Sadece belirlenen threshold'un üzerindeki "yelek" tahminlerini göster
                    if conf >= CONFIDENCE_THRESHOLD and "vest" in label.lower():
                        x, y, w, h = pred['x'], pred['y'], pred['width'], pred['height']
                        x1, y1 = int(x - w/2), int(y - h/2)
                        x2, y2 = int(x + w/2), int(y + h/2)

                        # Görselleştirme (Yeşil Kutu)
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
                        cv2.putText(frame, f"YETKILI: {conf:.2f}", (x1, y1-10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        except Exception as e:
            print(f"Hata: {e}")

    out.write(frame)
    frame_idx += 1
    if frame_idx % 50 == 0: print(f"İşlenen kare: {frame_idx}")

cap.release()
out.release()
print(f"✅ İşlem tamamlandı! Eşik değeri: {CONFIDENCE_THRESHOLD}")

İşlenen kare: 50
İşlenen kare: 100
✅ İşlem tamamlandı! Eşik değeri: 0.85


In [1]:
import cv2
import base64
import requests
import numpy as np
from ultralytics import YOLO

# 1. AYARLAR
API_KEY = "uwbLCWSnHV2geuphu3jv"
ROBO_MODEL = "safety-vests/14"
VIDEO_PATH = "test_yetkili.mp4"
CONF_THRESHOLD = 0.85  # İstediğin %85 eşiği

# Modeli yükle (Nano - en hafif sürüm)
model = YOLO('yolov8n.pt') 

cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("Hata: Video dosyası bulunamadı!")
    exit()

# Video özelliklerini al
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('denetim_85_threshold.mp4', 
                       cv2.VideoWriter_fourcc(*'mp4v'), 
                       fps, (w, h))

print(f"🚀 Analiz başladı. Threshold: {CONF_THRESHOLD}")

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Her 5 karede bir işlem yaparak CPU'yu rahatlatıyoruz
    if frame_idx % 5 == 0:
        # İnsanları tespit et (imgsz=320 çökme riskini azaltır)
        results = model.predict(frame, conf=0.5, classes=[0], imgsz=320, verbose=False)
        
        for box in results[0].boxes:
            coords = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, coords)
            
            # Kişiyi kırp ve Roboflow'a sor
            person_crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
            
            is_authorized = False
            if person_crop.size > 0:
                try:
                    # Görüntü hazırlama
                    _, buffer = cv2.imencode('.jpg', person_crop)
                    img_str = base64.b64encode(buffer).decode('utf-8')
                    
                    # API Sorgusu
                    url = f"https://detect.roboflow.com/{ROBO_MODEL}?api_key={API_KEY}"
                    response = requests.post(url, data=img_str, timeout=3).json()
                    
                    # Tahminleri kontrol et
                    predictions = response.get('predictions', [])
                    for p in predictions:
                        # Hem sınıf 'vest' olmalı hem de senin istediğin 0.85 threshold'u geçmeli
                        if p['class'].lower() == "vest" and p['confidence'] >= CONF_THRESHOLD:
                            is_authorized = True
                            break
                except Exception as e:
                    pass # İnternet hatalarını sessizce geç

            # ÇİZİM
            color = (0, 255, 0) if is_authorized else (0, 0, 255)
            label = f"YETKILI (%{CONF_THRESHOLD*100})" if is_authorized else "YETKISIZ"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    out.write(frame)
    frame_idx += 1
    if frame_idx % 50 == 0:
        print(f"İşlenen kare: {frame_idx}")

cap.release()
out.release()
print("✅ Analiz bitti. 'denetim_85_threshold.mp4' dosyasını kontrol edebilirsin.")

🚀 Analiz başladı. Threshold: 0.85
İşlenen kare: 50
İşlenen kare: 100
✅ Analiz bitti. 'denetim_85_threshold.mp4' dosyasını kontrol edebilirsin.


In [3]:
import cv2
import base64
import requests
import numpy as np
from ultralytics import YOLO

# 1. AYARLAR
API_KEY = "uwbLCWSnHV2geuphu3jv"
ROBO_MODEL = "safety-vests/14"
VIDEO_PATH = "test_yetkili.mp4"
PERSON_CONF = 0.30     # İnsan yakalama eşiği (Düşürüldü - Daha hassas)
VEST_CONF = 0.80       # Yelek onay eşiği (İstediğin yüksek değer)

# 'yolov8s.pt' daha iyi tespit yapar, 'n' zayıf kalıyorsa 's' deneyelim
model = YOLO('yolov8n.pt') 

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('denetim_hassas.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

print(f"🚀 Hassas analiz başladı. Kişi Eşiği: {PERSON_CONF}, Yelek Eşiği: {VEST_CONF}")

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # İşlemciyi yormamak için 3 karede bir analiz (Akıcılık için ideal)
    if frame_idx % 3 == 0:
        # imgsz=640 yaptık ki uzak personelleri de görebilsin
        results = model.predict(frame, conf=PERSON_CONF, classes=[0], imgsz=640, verbose=False)
        
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            # Kişiyi güvenli sınırlar içinde kırp
            person_crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
            
            is_authorized = False
            if person_crop.size > 0:
                try:
                    _, buffer = cv2.imencode('.jpg', person_crop)
                    img_str = base64.b64encode(buffer).decode('utf-8')
                    url = f"https://detect.roboflow.com/{ROBO_MODEL}?api_key={API_KEY}"
                    
                    # API Sorgusu
                    response = requests.post(url, data=img_str, timeout=2).json()
                    predictions = response.get('predictions', [])
                    
                    # Yelek kontrolü
                    for p in predictions:
                        if p['class'].lower() == "vest" and p['confidence'] >= VEST_CONF:
                            is_authorized = True
                            break
                except: pass

            # ÇİZİM
            color = (0, 255, 0) if is_authorized else (0, 0, 255)
            # Box'ı biraz daha kalınlaştırarak görünürlüğü artırdım
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 4)
            cv2.putText(frame, f"ID:{frame_idx} {'YETKILI' if is_authorized else 'YETKISIZ'}", 
                        (x1, y1 - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    out.write(frame)
    frame_idx += 1
    if frame_idx % 30 == 0: print(f"Kare {frame_idx} işleniyor...")

cap.release()
out.release()
print("✅ İşlem bitti. 'denetim_hassas.mp4' dosyasını kontrol et.")

🚀 Hassas analiz başladı. Kişi Eşiği: 0.3, Yelek Eşiği: 0.8
Kare 30 işleniyor...
Kare 60 işleniyor...
Kare 90 işleniyor...
Kare 120 işleniyor...
✅ İşlem bitti. 'denetim_hassas.mp4' dosyasını kontrol et.


In [1]:
import cv2
import base64
import requests
import numpy as np
from ultralytics import YOLO

# 1. AYARLAR
API_KEY = "uwbLCWSnHV2geuphu3jv"
ROBO_MODEL = "safety-vests/14"
VIDEO_PATH = "test_yetkili.mp4"
VEST_CONF = 0.60  # Yelek yakalama eşiği

# Modeli sadece predict modunda kullanıyoruz (Tracking kapalı - Çökme engellendi)
model = YOLO('yolov8n.pt') 

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('denetim_manuel_id.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

# Hafıza Yönetimi
authorized_ids = set()
person_history = {} # {id: (x, y)} son konumları tutar
next_id = 1

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Sadece her 2 karede bir işlem yapalım
    if frame_idx % 2 == 0:
        results = model.predict(frame, conf=0.4, imgsz=640, classes=[0], verbose=False)
        
        current_frame_persons = []
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2 # Merkez noktası
            
            # --- MANUEL ID ATAMA (Mesafe Bazlı) ---
            assigned_id = None
            min_dist = 50 # 50 pikselden yakınsa aynı kişidir
            
            for pid, last_pos in person_history.items():
                dist = np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2)
                if dist < min_dist:
                    assigned_id = pid
                    min_dist = dist
                    break
            
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            current_frame_persons.append((assigned_id, (x1, y1, x2, y2)))

            # --- YETKİ KONTROLÜ (15 karede bir yeni ID'ler için) ---
            if assigned_id not in authorized_ids and frame_idx % 15 == 0:
                person_crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if person_crop.size > 0:
                    try:
                        _, buffer = cv2.imencode('.jpg', person_crop)
                        img_str = base64.b64encode(buffer).decode('utf-8')
                        resp = requests.post(f"https://detect.roboflow.com/{ROBO_MODEL}?api_key={API_KEY}", 
                                             data=img_str, timeout=1).json()
                        if any(p['class'].lower() == "vest" and p['confidence'] >= VEST_CONF for p in resp.get('predictions', [])):
                            authorized_ids.add(assigned_id)
                    except: pass

            # --- ÇİZİM ---
            is_auth = assigned_id in authorized_ids
            color = (0, 255, 0) if is_auth else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.putText(frame, f"ID:{assigned_id} {'YETKILI' if is_auth else 'PERSONEL'}", 
                        (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    out.write(frame)
    frame_idx += 1
    # Çok eski personelleri bellekten sil
    if len(person_history) > 20: person_history = {k: v for i, (k, v) in enumerate(person_history.items()) if i > 5}

cap.release()
out.release()
print("✅ Analiz ÇÖKMEDEN tamamlandı!")

✅ Analiz ÇÖKMEDEN tamamlandı!


In [2]:
import cv2
import base64
import requests
import numpy as np
from ultralytics import YOLO

# --- AYARLAR ---
API_KEY = "uwbLCWSnHV2geuphu3jv"
ROBO_MODEL = "safety-vests/14"
VIDEO_PATH = "test_yetkili.mp4"
VEST_THRESHOLD = 0.70  # Yelek tespiti için eşik

# Modeller
model = YOLO('yolov8n.pt') 

# Hafıza
authorized_ids = set() # Yelekli olduğu onaylanan ID'ler
person_history = {}    # {id: (cx, cy)}
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('yetki_oncelikli_denetim.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Performans için her 2 karede bir derin analiz
    if frame_idx % 2 == 0:
        results = model.predict(frame, conf=0.4, imgsz=640, classes=[0], verbose=False)
        
        current_frame_data = [] # (id, x1, y1, x2, y2)
        
        # 1. PERSON TESPİTİ VE ID ATAMA
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            assigned_id = None
            min_dist = 60 # Mesafe toleransı
            
            for pid, last_pos in person_history.items():
                dist = np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2)
                if dist < min_dist:
                    assigned_id = pid
                    min_dist = dist
                    break
            
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            current_frame_data.append((assigned_id, x1, y1, x2, y2))

        # 2. YETKİ KONTROLÜ VE ÇİZİM (ID BAZLI)
        for pid, x1, y1, x2, y2 in current_frame_data:
            # Eğer ID yetkili değilse ve her 15 karede bir sırası gelmişse API'ye sor
            if pid not in authorized_ids and frame_idx % 15 == 0:
                crop = frame[max(0,y1):min(h,y2), max(0,x1):min(w,x2)]
                if crop.size > 0:
                    try:
                        _, buffer = cv2.imencode('.jpg', crop)
                        img_base64 = base64.b64encode(buffer).decode('utf-8')
                        resp = requests.post(f"https://detect.roboflow.com/{ROBO_MODEL}?api_key={API_KEY}", 
                                             data=img_base64, timeout=1).json()
                        
                        # API'den gelen cevapta 'vest' (yelek) var mı?
                        if any(p['class'].lower() == "vest" and p['confidence'] >= VEST_THRESHOLD for p in resp.get('predictions', [])):
                            authorized_ids.add(pid)
                    except: pass

            # --- TEK BİR ÇİZİM MANTIĞI ---
            # Yetkili ise YEŞİL, değilse KIRMIZI (Burada "ezme" durumu bitti)
            if pid in authorized_ids:
                color = (0, 255, 0) # Yeşil
                label = f"ID:{pid} YETKILI"
            else:
                color = (0, 0, 255) # Kırmızı
                label = f"ID:{pid} PERSONEL"

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            # Etiket arka planı (Okunabilirlik için)
            cv2.rectangle(frame, (x1, y1 - 25), (x1 + 140, y1), color, -1)
            cv2.putText(frame, label, (x1 + 5, y1 - 8), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    out.write(frame)
    frame_idx += 1
    
    # Bellek temizliği (100 frame'den eski olanları siler)
    if len(person_history) > 50:
        person_history = {k: v for i, (k, v) in enumerate(person_history.items()) if i > 10}

cap.release()
out.release()
print("✅ Analiz tamamlandı. Çizimlerde yetki önceliği sağlandı.")

✅ Analiz tamamlandı. Çizimlerde yetki önceliği sağlandı.


In [3]:
import cv2
import base64
import requests
import numpy as np
from ultralytics import YOLO

# --- AYARLAR ---
API_KEY = "uwbLCWSnHV2geuphu3jv"
ROBO_MODEL = "safety-vests/14"
VIDEO_PATH = "test_yetkili.mp4"
VEST_THRESHOLD = 0.75  # Yelek hassasiyeti

# Modeli yükle
model = YOLO('yolov8n.pt') 

# Hafıza Yönetimi
authorized_ids = set() # Yeleği onaylanmış ID'ler
person_history = {}    # {id: (cx, cy)} son konumlar
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('yetki_oncelikli_2.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Performans ve stabilite için her 2 karede bir derin analiz
    if frame_idx % 2 == 0:
        # İnsanları tespit et
        results = model.predict(frame, conf=0.4, imgsz=640, classes=[0], verbose=False)
        
        current_frame_detections = [] # (id, x1, y1, x2, y2)
        
        # 1. ADIM: TESPİT VE ID ATAMA (Çizim yok)
        for box in results[0].boxes:
            coords = box.xyxy[0].tolist()
            x1, y1, x2, y2 = map(int, coords)
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # Mesafe bazlı basit takip (Tracking)
            assigned_id = None
            min_dist = 60 
            for pid, last_pos in person_history.items():
                dist = np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2)
                if dist < min_dist:
                    assigned_id = pid
                    min_dist = dist
                    break
            
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            current_frame_detections.append((assigned_id, x1, y1, x2, y2))

        # 2. ADIM: YETKİ KONTROLÜ VE ÖNCELİKLİ ÇİZİM
        for pid, x1, y1, x2, y2 in current_frame_detections:
            
            # Eğer bu ID henüz yetkili değilse, 15 karede bir Roboflow'a sor
            if pid not in authorized_ids and frame_idx % 15 == 0:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if crop.size > 0:
                    try:
                        _, buffer = cv2.imencode('.jpg', crop)
                        img_str = base64.b64encode(buffer).decode('utf-8')
                        resp = requests.post(f"https://detect.roboflow.com/{ROBO_MODEL}?api_key={API_KEY}", 
                                             data=img_str, timeout=1).json()
                        
                        if any(p['class'].lower() == "vest" and p['confidence'] >= VEST_THRESHOLD for p in resp.get('predictions', [])):
                            authorized_ids.add(pid)
                    except: pass

            # 3. ADIM: TEK BİR ETİKET BASMA (EZMEYİ ÖNLEYEN KISIM)
            # Eğer ID yetkili listesindeyse YEŞİL, değilse KIRMIZI çiz.
            if pid in authorized_ids:
                color = (0, 255, 0) # Yeşil
                label = f"ID:{pid} YETKILI"
                thickness = 4 # Yetkiliyi daha belirgin yap
            else:
                color = (0, 0, 255) # Kırmızı
                label = f"ID:{pid} PERSONEL"
                thickness = 2

            # Sadece tek bir box ve tek bir label çiziyoruz
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            
            # Yazı arka planı
            (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame, (x1, y1 - 30), (x1 + label_w + 10, y1), color, -1)
            cv2.putText(frame, label, (x1 + 5, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    out.write(frame)
    frame_idx += 1
    
    # Bellek Yönetimi (Eski konumları sil)
    if len(person_history) > 40:
        person_history = {k: v for i, (k, v) in enumerate(person_history.items()) if i > 10}

cap.release()
out.release()
print("✅ Analiz bitti. Yetki önceliği atandı, çakışan kutular giderildi.")

✅ Analiz bitti. Yetki önceliği atandı, çakışan kutular giderildi.


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- AYARLAR ---
VIDEO_PATH = "test_yetkili.mp4"
GREEN_RATIO_THRESHOLD = 0.08  # Üst gövdenin en az %8'i yeşil olmalı
PERSON_CONF = 0.40

# Modeli yükle (Nano - en hızlısı)
model = YOLO('yolov8n.pt') 

def is_wearing_green_vest(person_img):
    if person_img is None or person_img.size == 0:
        return False
    
    # 1. Sadece Üst Gövdeye Odaklan (Kutunun üst %10 ile %45 arası)
    h, w = person_img.shape[:2]
    upper_body = person_img[int(h*0.1):int(h*0.45), :]
    
    # 2. HSV Renk Uzayına Geç (Işığa karşı daha dirençli)
    hsv = cv2.cvtColor(upper_body, cv2.COLOR_BGR2HSV)
    
    # 3. Fosforlu Yeşil/Sarı Yelek Aralığı (Fabrika yelekleri için optimize)
    # Bu değerleri yeleğinin tonuna göre hafifçe oynatabilirsin
    lower_green = np.array([35, 50, 50])   # Alt sınır
    upper_green = np.array([90, 255, 255]) # Üst sınır
    
    # 4. Maskeleme ve Oran Hesaplama
    mask = cv2.inRange(hsv, lower_green, upper_green)
    green_pixel_count = np.sum(mask > 0)
    total_pixels = mask.size
    ratio = green_pixel_count / total_pixels
    
    return ratio > GREEN_RATIO_THRESHOLD

# --- VİDEO İŞLEME ---
cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('renk_analizli_denetim.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Her karede analiz yapabiliriz çünkü renk analizi çok hızlıdır
    results = model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
    
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        
        # Kişiyi kes
        person_crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
        
        # Renk Analizi Yap
        authorized = is_wearing_green_vest(person_crop)
        
        # Görselleştirme
        if authorized:
            color = (0, 255, 0) # Yeşil
            label = "YETKILI"
        else:
            color = (0, 0, 255) # Kırmızı
            label = "PERSONEL"
            
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
        cv2.rectangle(frame, (x1, y1 - 30), (x1 + 110, y1), color, -1)
        cv2.putText(frame, label, (x1 + 5, y1 - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    out.write(frame)
    frame_idx += 1
    if frame_idx % 50 == 0: print(f"{frame_idx}. kare işlendi...")

cap.release()
out.release()
print("✅ İşlem bitti! Roboflow yerine yerel renk analizi kullanıldı.")

50. kare işlendi...
100. kare işlendi...
✅ İşlem bitti! Roboflow yerine yerel renk analizi kullanıldı.


In [5]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- AYARLAR ---
VIDEO_PATH = "test_yetkili.mp4"
GREEN_RATIO_THRESHOLD = 0.06  # Yelek hassasiyeti (Gerekirse düşür/artır)
PERSON_CONF = 0.35            # İnsan yakalama hassasiyeti

# Modeli yükle
model = YOLO('yolov8n.pt') 

# Takip ve Yetki Hafızası
authorized_ids = set() # Yeleği bir kez onaylanan ID'ler buraya girer
person_history = {}    # {id: (merkez_x, merkez_y)}
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
out = cv2.VideoWriter('takip_ve_renk_analizi.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 20.0, (w, h))

def check_green_locally(person_img):
    """Görüntüdeki üst gövdede yeşil yoğunluğunu ölçer."""
    if person_img is None or person_img.size == 0: return False
    img_h, img_w = person_img.shape[:2]
    # Sadece omuz/göğüs bölgesi (Üst %10-45 arası)
    upper = person_img[int(img_h*0.1):int(img_h*0.45), :]
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    # Fosforlu Yeşil/Sarı Aralığı
    lower_green = np.array([30, 45, 45])   # Biraz daha genişlettim (sarıya yakın)
    upper_green = np.array([95, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Analizi 2 karede bir yapıyoruz (Stabilite için)
    if frame_idx % 2 == 0:
        results = model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
        
        current_frame_detections = []
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # --- MANUEL ID ATAMA ---
            assigned_id = None
            min_dist = 65 # Mesafe eşiği
            for pid, last_pos in person_history.items():
                dist = np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2)
                if dist < min_dist:
                    assigned_id = pid
                    min_dist = dist
                    break
            
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            person_history[assigned_id] = (cx, cy)
            current_frame_detections.append((assigned_id, x1, y1, x2, y2))

        # --- YETKİ KONTROLÜ VE ÇİZİM ---
        for pid, x1, y1, x2, y2 in current_frame_detections:
            # Eğer ID daha önce yetkili olarak işaretlenmediyse renk analizi yap
            if pid not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop):
                    authorized_ids.add(pid)

            # ÇİZİM: Yetkili hafızadaysa YEŞİL, değilse KIRMIZI
            is_auth = pid in authorized_ids
            color = (0, 255, 0) if is_auth else (0, 0, 255)
            label = f"ID:{pid} YETKILI" if is_auth else f"ID:{pid} PERSONEL"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.rectangle(frame, (x1, y1 - 25), (x1 + 150, y1), color, -1)
            cv2.putText(frame, label, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    out.write(frame)
    frame_idx += 1
    
    # Çok eski personelleri bellekten temizle
    if len(person_history) > 30:
        person_history = {k: v for i, (k, v) in enumerate(person_history.items()) if i > 5}

cap.release()
out.release()
print("✅ Manuel Takip + Renk Analizi tamamlandı.")

✅ Manuel Takip + Renk Analizi tamamlandı.


In [7]:
import cv2
import numpy as np
from ultralytics import YOLO

# --- AYARLAR ---
VIDEO_PATH = "test_yetkili.mp4"
GREEN_RATIO_THRESHOLD = 0.06  # Yelek hassasiyeti
PERSON_CONF = 0.50            # İnsan yakalama hassasiyeti

model = YOLO('yolov8n.pt') 

# Hafıza Yönetimi
authorized_ids = set() 
person_history = {}    # {id: (cx, cy)}
last_boxes = {}        # {id: (x1, y1, x2, y2)} -> Kutuların sabit kalması için
next_id = 1

cap = cv2.VideoCapture(VIDEO_PATH)
w, h = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('sabit_denetim.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

def check_green_locally(person_img):
    if person_img is None or person_img.size == 0: return False
    img_h, img_w = person_img.shape[:2]
    upper = person_img[int(img_h*0.1):int(img_h*0.45), :]
    hsv = cv2.cvtColor(upper, cv2.COLOR_BGR2HSV)
    lower_green = np.array([30, 40, 40])
    upper_green = np.array([95, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    return (np.sum(mask > 0) / mask.size) > GREEN_RATIO_THRESHOLD

frame_idx = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # Analizi her karede yapıyoruz ama çizimi hafızadan besliyoruz
    results = model.predict(frame, conf=PERSON_CONF, imgsz=640, classes=[0], verbose=False)
    
    current_frame_ids = []
    
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            
            # --- MANUEL ID TAKİBİ ---
            assigned_id = None
            min_dist = 80 
            for pid, last_pos in person_history.items():
                dist = np.sqrt((cx - last_pos[0])**2 + (cy - last_pos[1])**2)
                if dist < min_dist:
                    assigned_id = pid
                    min_dist = dist
                    break
            
            if assigned_id is None:
                assigned_id = next_id
                next_id += 1
            
            # Konum ve Kutu bilgilerini güncelle
            person_history[assigned_id] = (cx, cy)
            last_boxes[assigned_id] = (x1, y1, x2, y2)
            current_frame_ids.append(assigned_id)

            # Yetki kontrolü (Hafızada yoksa bak)
            if assigned_id not in authorized_ids:
                crop = frame[max(0, y1):min(h, y2), max(0, x1):min(w, x2)]
                if check_green_locally(crop):
                    authorized_ids.add(assigned_id)

    # --- ÇİZİM AŞAMASI (HAFIZADAN ÇİZ) ---
    # Sadece o karede tespit edilenleri değil, hafızadaki aktif kutuları çiz
    # Böylece model bir kare kaçırsa bile kutu kaybolmaz
    for pid in list(last_boxes.keys()):
        # Eğer kişi uzun süredir tespit edilemiyorsa hafızadan sil (Örn: 10 kare)
        # Şimdilik sadece o karede olanları veya çok yakın geçmiştekileri çizelim
        if pid in current_frame_ids:
            x1, y1, x2, y2 = last_boxes[pid]
            is_auth = pid in authorized_ids
            color = (0, 255, 0) if is_auth else (0, 0, 255)
            label = f"ID:{pid} YETKILI" if is_auth else f"ID:{pid} PERSONEL"
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
            cv2.rectangle(frame, (x1, y1 - 25), (x1 + 150, y1), color, -1)
            cv2.putText(frame, label, (x1 + 5, y1 - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    out.write(frame)
    frame_idx += 1
    
    # Hafıza temizliği (Görüntüden çıkanları silmek için)
    if len(person_history) > 30:
        active_ids = set(current_frame_ids)
        person_history = {k: v for k, v in person_history.items() if k in active_ids}
        last_boxes = {k: v for k, v in last_boxes.items() if k in active_ids}

cap.release()
out.release()
print("✅ Sarsıntısız takip tamamlandı. Boxlar artık sabit kalmalı.")

✅ Sarsıntısız takip tamamlandı. Boxlar artık sabit kalmalı.
